# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-2 #VGG 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients
tol=threshold

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

In [3]:
molecule='lih'
basis = 'sto-3g'
R=1.5

### LiH in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_2$ in minimal basis at $R=1.5$ (Angstrom). For comparison, we can run CISD to obtain the true ground state energy.


In [4]:
xyz_data = get_molecular_data(molecule, geometry=R, xyz_format=True)
basis='sto-3g'

hm = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*hm.n_orbitals))

e_0 = hm.compute_energy(method='cisd')

print('CISD energy: {}'.format(e_0))

Number of spin-orbitals (qubits): 12 

CISD energy: -7.882433089189864


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [5]:
H = hm.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = hm.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", 
                     initial_values={k:0.0 for k in E.extract_variables()}, tol=tol) #VGG 1e-6

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 631 terms


Number of UCCSD amplitudes: 6 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 192 expectationvalues

active variables : 6

E=-7.86335762  angles= {(2, 1, 5, 1): 0.0, (2, 1, 2, 1): 0.0, (5, 1, 5, 1): 0.0, (3, 1, 3, 1): 0.0, (5, 1, 2, 1): 0.0, (4, 1, 4, 1): 0.0}  samples= None
E=-7.85955830  angles= {(2, 1, 5, 1): -0.06699275970458984, (2, 1, 2, 1): -0.024281024932861328, (5, 1, 5, 1): -0.24590444564819336, (3, 1, 3, 1): -0.0479731559753418, (5, 1, 2, 1): -0.06699275970458984, (4, 1, 4, 1): -0.0479731559753418}  samples= None
E=-7.88120928  angles= {(2, 1, 5, 1): -0.03184077673332027, (2, 1, 2, 1): -0.011540451492856645, (5, 1, 5, 1): -0.11687514570441819, (3, 1, 3, 1): -0.022801009472355773, (5, 1, 2, 1): -0.03184077673332027, (4, 1, 4, 

We see that the converged UCCSD energy is in exact agreement with the CISD energy, as expected for a $4$-electron system. 


## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### LiH in STO-3G basis

Below we perform the entangler screening protocol for H2 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [6]:
basis='sto-3g'

xyz_data = get_molecular_data(molecule, geometry=R, xyz_format=True)

h2 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

hf_reference = hf_occ(2*h2.n_orbitals, h2.n_electrons)

H = h2.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h2.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h2.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 631 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.123
2 : 0.0335
3 : 0.0335
4 : 0.024
5 : 0.024
6 : 0.0215
7 : 0.0121

Selected entanglers:
1.0 [X2 Y3 X10 X11]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [7]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h2.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=tol)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 50 expectationvalues

active variables : 25

E=-6.25839976  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, beta_8: 0.0, gamma_8: 0.0, beta_9: 0.0, gamma_9: 0.0, beta_10: 3.141592653589793, gamma_10: 0.0, beta_11: 3.141592653589793, gamma_11: 0.0, tau_0: 0.0}  samples= None
Optimization terminated successfully.
         Current function value: -6.258400
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1

Obtained QCC energy (1 entanglers): -6.258399755644748


## 6-31G basis Qubit Coupled Cluster (QCC)

### LiH in 6-31G basis

Now let us try a larger problem of H$_2$O in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [8]:
xyz_data = get_molecular_data(molecule, geometry=R, xyz_format=True)

basis = '6-31g'
#active = {'B1':[0,1], 'A1':[2,3]}
hm = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis)#, active_orbitals = active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*hm.n_orbitals))

e0 = hm.compute_energy(method='cisd')

print('CISD energy: {}'.format(e0))

Number of spin-orbitals (qubits): 22 

CISD energy: -7.995374283465799


We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [ ]:
H = hm.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = hm.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", 
                     initial_values={k:0.0 for k in E.extract_variables()}, tol=tol) #VGG tol was 1e-4 

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 6870 terms


Number of UCCSD amplitudes: 22 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 704 expectationvalues

active variables : 22

E=-7.97685913  angles= {(5, 1, 2, 1): 0.0, (3, 1, 7, 1): 0.0, (4, 1, 8, 1): 0.0, (9, 1, 10, 1): 0.0, (10, 1, 9, 1): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (5, 1, 9, 1): 0.0, (9, 1, 2, 1): 0.0, (8, 1, 8, 1): 0.0, (4, 1, 4, 1): 0.0, (10, 1, 5, 1): 0.0, (8, 1, 4, 1): 0.0, (9, 1, 9, 1): 0.0, (2, 1, 5, 1): 0.0, (5, 1, 5, 1): 0.0, (5, 1, 10, 1): 0.0, (7, 1, 3, 1): 0.0, (9, 1, 5, 1): 0.0, (10, 1, 10, 1): 0.0, (2, 1, 9, 1): 0.0, (7, 1, 7, 1): 0.0}  samples= None


We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [ ]:
#n=10
#result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

We see that the QCC energy converged to the FCI energy with only a single entangler! 

### LiH in 6-31G basis

Let us move on to the problem of H$_2$O in 6-31G basis using the same active space as the UCCSD example.

In [ ]:
basis = '6-31g'

xyz_data = get_molecular_data(molecule, geometry=R, xyz_format=True)

#active = {'B1':[0,1], 'A1':[2,3]}
hm = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis )#, active_orbitals = active)
hf_reference = hf_occ(2*hm.n_orbitals, hm.n_electrons)


H = hm.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*hm.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*hm.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

With the 6 chosen entanglers, we can then run the QCC VQE simulation.

In [ ]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*hm.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=tol)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

We obtain chemical accuracy for water near equilibrium geometry with only 6 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [ ]:
n = 5 #VGG10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=tol, n=n) #VGG tol=1e-2

print('\nObtained QCC energy ({} entanglers): {:6.3f}'.format(len(entanglers), result))

In [ ]:
print('\nObtained QCC energy ({} entanglers): {:6.3f}'.format(len(entanglers), result))